In [1]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


In [0]:
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
%%capture
!apt-get install libav-tools -y

In [0]:
import gc

In [0]:
from keras.preprocessing import image

In [0]:
!pip install memory_profiler

    100% |████████████████████████████████| 40kB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler


In [0]:
import numpy as np
#from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import librosa
import librosa.display

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = './gdrive/My Drive/OST_Project/working/'

In [0]:
image_height = 217
image_width = 223

In [12]:
 
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(image_height,image_width,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 217, 223, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 217, 223, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 215, 221, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 215, 221, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 107, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
model.load_weights(root_path +"weights_best2.hdf5")

In [0]:
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [0]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
validation_dir = root_path + 'test'

In [17]:
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_height, image_width),
        batch_size=10,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

Found 308 images belonging to 10 classes.
31/30 [==============================] - 112s 4s/step
No of errors = 109/308


In [0]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = root_path + 'images/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [0]:
Data_dir=np.array(glob(root_path+'test_sounds/*'))

In [0]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [0]:
%%memit 

for file in Data_dir:
    #print(file)
    filename,name = file,file.split('/')[-1].split('.')[0]
    #print(filename,name)
    create_spectrogram(filename,name)

peak memory: 1838.41 MiB, increment: 0.24 MiB


In [0]:
gc.collect()

0

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
test_dir = root_path + 'images'

In [0]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(image_height, image_width))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [0]:
img_dir=np.array(glob(root_path+'images/*'))

In [0]:
%%memit 

for file in img_dir:
   
    new_image = load_image(file)

    # check prediction
    pred = model.predict(new_image)
    print(pred)
    predicted_class = np.argmax(pred,axis=1)
    print(idx2label[predicted_class[0]])

[[7.0434438e-26 3.1980082e-32 1.0298103e-20 1.0000000e+00 2.9120112e-15
  3.8082251e-28 6.5875237e-23 6.8264240e-27 2.0221763e-22 3.4902010e-20]]
dog_bark
[[3.4169452e-18 7.6938977e-24 5.2847236e-14 1.0000000e+00 4.1430735e-12
  4.6158047e-18 7.4839749e-15 1.7828340e-17 1.6427189e-12 1.0477842e-13]]
dog_bark
peak memory: 1013.34 MiB, increment: 101.82 MiB
